## Chapter 12: kNN

Trying to predict how I'm going to vote in next presidential election. If you know nothing else about me (+ if you have the data), 1 sensible approach = look @ how my neighbors are planning to vote. 

Ex: Downtown Seattle, = invariably planning to vote Democrat, which suggests “Democrat” =  good guess for me as well.

Now imagine you know *more* about me than just geography = age, income, # of kids, etc. To the extent my behavior is influenced/characterized by those things, looking just @ neighbors who are *close to me among all those dimensions* seems likely to be an even better predictor than looking @ all neighbors = **nearest neighbors classification.**

### The Model

Nearest neighbors = 1 of the simplest predictive models there is as it *makes no
mathematical assumptions* + doesn’t require any sort of heavy machinery. 

Only things it requires are:

* Some notion of distance
* Assumption that DP's that're close to one another = similar

Many techniques look @ the data set *as a whole* in order to learn patterns in the data. Nearest neighbors, on the other hand, *consciously neglects a lot of info*, since the prediction for *each new point depends only on the handful of DPs closest to it*.

What’s more, nearest neighbors is probably *NOT* going to help understand drivers
of whatever phenomenon you’re looking at. Predicting votes based on neighbors’ votes doesn’t tell you much about what causes me to vote the way I do, whereas some alternative model that predicted vote based on income + marital status, for example, very well might.

Generally, we have some DPs + a corresponding set of labels (could be T/F or categories). In our case, the DPs = vectors, which means we can use `distance`

In [1]:
def vector_subtract(v,w):
    """Adds corresponding elements"""
    return[v_i - w_i
          for v_i, w_i in zip(v,w)]

# perform dot product = sum of component-wise products
def dot_prod(v,w):
    # for each component in vectors the zip list, multiply it by the
    # corresponding-indexed element in the other vectors in the zipped list
    return sum(v_i*w_i           
              for v_i,w_i in zip(v,w))

## use dot product to easily compute SUM OF SQUARES of a vector
def vector_SS(v):
    """v1*v1 + ... + vn*vn"""
    return dot_prod(v,v)

## compute distance between 2 vectors == sqrt[(v1 - w1)**2 + ... + (vn-wn)**2]
def squared_distance(v,w):
    # get squared distance from each element-wise difference
    return vector_SS(vector_subtract(v,w))

def distance(v,w):
    from math import sqrt
    
    return sqrt(squared_distance(v,w))

Let’s say we’ve picked 3 for `k` + we want to classify some new DP, so we find the `k` nearest labeled DPs + let them vote on the new output. To do this, we’ll need a function that counts votes. One possibility is:

In [2]:
def raw_majority_vote(labels):
    from collections import Counter
    
    votes = Counter(labels)
    # get most common (1) label + grab 1st element in tupel = the label itself [0]
    winner, _ = votes.most_common(1)[0]
    return winner

But this doesn’t do anything intelligent w/ ties. 

* Ex: Rating movies + the 5 nearest movies are rated: {G, G, PG, PG, R}. 
* G + PG have 2 votes, so we have several options:
    * Pick 1 of the winners at random.
    * Weight votes by distance + pick the *weighted winner*
    * Reduce k until we find a unique winner
    
Implement the third:

In [3]:
# if tie, reduce k until unique winner found
def majority_vote(labels):
    """Assumes labels are ordered from nearest to farthest"""
    vote_counts = Counter(labels)
    # get most common (1) label + grab 1st element in tupel = the label itself [0]
    winner, winner_count = vote_counts.most_common(1)[0]
    
    # grab number of labels whose count = the most common count (highest count)
    num_winners = len([count
                      for count in vote_counts.values()
                      if count == winner_count])
    
    # if only 1 winner (unique), return
    # if not, decrease k (remove farthest neighbor) and repeat process
    if num_winners == 1:
        return winner
    else: 
        return majority_vote(labels[:-1])

This approach is sure to work eventually, since in the worst case we go all the way down
to just 1 label, at which point that 1 label wins. With this function it’s easy to create a classifier:

In [5]:
def knn_classify(k,labeled_points,new_point):
    """Each labeled point shuold be a pair = (point, label)"""
    # Order labeled points from nearest to farthest from given new point
    by_distance = sorted(labeled_points,
                         key = lambda point_label: distance(point_label[0], new_point))
    
    # Find labels for k closest
    knn_labels = [label for _, label in by_distance[:k]]
    
    # Let them vote
    return majority_vote(knn_labels)

In [9]:
#cities = [(-86.75,33.5666666666667,'Python'),(-88.25,30.6833333333333,'Python'),(-112.016666666667,33.4333333333333,'Java'),(-110.933333333333,32.1166666666667,'Java'),(-92.2333333333333,34.7333333333333,'R'),(-121.95,37.7,'R'),(-118.15,33.8166666666667,'Python'),(-118.233333333333,34.05,'Java'),(-122.316666666667,37.8166666666667,'R'),(-117.6,34.05,'Python'),(-116.533333333333,33.8166666666667,'Python'),(-121.5,38.5166666666667,'R'),(-117.166666666667,32.7333333333333,'R'),(-122.383333333333,37.6166666666667,'R'),(-121.933333333333,37.3666666666667,'R'),(-122.016666666667,36.9833333333333,'Python'),(-104.716666666667,38.8166666666667,'Python'),(-104.866666666667,39.75,'Python'),(-72.65,41.7333333333333,'R'),(-75.6,39.6666666666667,'Python'),(-77.0333333333333,38.85,'Python'),(-80.2666666666667,25.8,'Java'),(-81.3833333333333,28.55,'Java'),(-82.5333333333333,27.9666666666667,'Java'),(-84.4333333333333,33.65,'Python'),(-116.216666666667,43.5666666666667,'Python'),(-87.75,41.7833333333333,'Java'),(-86.2833333333333,39.7333333333333,'Java'),(-93.65,41.5333333333333,'Java'),(-97.4166666666667,37.65,'Java'),(-85.7333333333333,38.1833333333333,'Python'),(-90.25,29.9833333333333,'Java'),(-70.3166666666667,43.65,'R'),(-76.6666666666667,39.1833333333333,'R'),(-71.0333333333333,42.3666666666667,'R'),(-72.5333333333333,42.2,'R'),(-83.0166666666667,42.4166666666667,'Python'),(-84.6,42.7833333333333,'Python'),(-93.2166666666667,44.8833333333333,'Python'),(-90.0833333333333,32.3166666666667,'Java'),(-94.5833333333333,39.1166666666667,'Java'),(-90.3833333333333,38.75,'Python'),(-108.533333333333,45.8,'Python'),(-95.9,41.3,'Python'),(-115.166666666667,36.0833333333333,'Java'),(-71.4333333333333,42.9333333333333,'R'),(-74.1666666666667,40.7,'R'),(-106.616666666667,35.05,'Python'),(-78.7333333333333,42.9333333333333,'R'),(-73.9666666666667,40.7833333333333,'R'),(-80.9333333333333,35.2166666666667,'Python'),(-78.7833333333333,35.8666666666667,'Python'),(-100.75,46.7666666666667,'Java'),(-84.5166666666667,39.15,'Java'),(-81.85,41.4,'Java'),(-82.8833333333333,40,'Java'),(-97.6,35.4,'Python'),(-122.666666666667,45.5333333333333,'Python'),(-75.25,39.8833333333333,'Python'),(-80.2166666666667,40.5,'Python'),(-71.4333333333333,41.7333333333333,'R'),(-81.1166666666667,33.95,'R'),(-96.7333333333333,43.5666666666667,'Python'),(-90,35.05,'R'),(-86.6833333333333,36.1166666666667,'R'),(-97.7,30.3,'Python'),(-96.85,32.85,'Java'),(-95.35,29.9666666666667,'Java'),(-98.4666666666667,29.5333333333333,'Java'),(-111.966666666667,40.7666666666667,'Python'),(-73.15,44.4666666666667,'R'),(-77.3333333333333,37.5,'Python'),(-122.3,47.5333333333333,'Python'),(-89.3333333333333,43.1333333333333,'R'),(-104.816666666667,41.15,'Java')]
#knn_classify(5,cities,(-83,36,'R'))

In [10]:
sorted(cities,key = lambda point_label: distance(point_label[0],(-83,36,'R')))

NameError: name 'sqrt' is not defined